In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras import Sequential
import tensorflow.keras.layers as nn

from tensorflow import einsum
from einops import rearrange
from einops.layers.tensorflow import Rearrange

import math
from inspect import isfunction

from functools import partial
from tqdm import tqdm

import numpy as np
import os
import cv2
import pathlib
from glob import glob

import time
from tensorflow.python.data.experimental import AUTOTUNE
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import pandas as pd
import skimage
from skimage.metrics import structural_similarity
import time
from skimage import filters, img_as_ubyte,morphology,measure
from scipy.ndimage import label
import copy
import random

In [ ]:
################数据导入###################
degree5_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-200-5.csv").iloc[1:,1:]
degree5_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-300-5.csv").iloc[1:,1:]
degree5_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\5-400-5.csv").iloc[1:,1:]
degree15_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-200-5.csv").iloc[1:,1:]
degree15_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-300-5.csv").iloc[1:,1:]
degree15_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\15-400-5.csv").iloc[1:,1:]
degree25_200_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-200-5.csv").iloc[1:,1:]
degree25_300_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-300-5.csv").iloc[1:,1:]
degree25_400_5 = pd.read_csv(r"D:\AI in NTU\PINN-fatigue\论文数据汇总\5hzC-S数据\25-400-5.csv").iloc[1:,1:] ###测试

step_max = int(np.array([degree5_200_5.max(),degree5_300_5.max(),degree5_400_5.max(),
            degree15_200_5.max(),degree15_300_5.max(),degree15_400_5.max(),
            degree25_200_5.max(),degree25_300_5.max(),degree25_400_5.max(),]).max())+500
####建立训练集
###lenght=15, 预测1个,随即时间间隔+物理约束
####5,15,25三类，200，300，400三类
X_data = [] 
X_data_increase = []
X_data_factor = [] 
Y_data = []

########5度###############
for i in range (len(degree5_200_5)-15):
    X_data.append(np.transpose([np.array(degree5_200_5.iloc[i:i+15,1])/step_max,np.array(degree5_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[1,0,0]])
    X_data_increase.append(degree5_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_200_5.iloc[i+15,0])

for i in range (len(degree5_300_5)-15):
    X_data.append(np.transpose([np.array(degree5_300_5.iloc[i:i+15,1])/step_max,np.array(degree5_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[0,1,0]])
    X_data_increase.append(degree5_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_300_5.iloc[i+15,0])

for i in range (len(degree5_400_5)-15):
    X_data.append(np.transpose([np.array(degree5_400_5.iloc[i:i+15,1])/step_max,np.array(degree5_400_5.iloc[i:i+15,0])]))
    X_data_factor.append([[1,0,0],[0,0,1]])
    X_data_increase.append(degree5_400_5.iloc[i+15,1]/step_max)
    Y_data.append(degree5_400_5.iloc[i+15,0])

########15度###############
for i in range (len(degree15_200_5)-15):
    X_data.append(np.transpose([np.array(degree15_200_5.iloc[i:i+15,1])/step_max,np.array(degree15_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[1,0,0]])
    X_data_increase.append(degree15_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_200_5.iloc[i+15,0])

for i in range (len(degree15_300_5)-15):
    X_data.append(np.transpose([np.array(degree15_300_5.iloc[i:i+15,1])/step_max,np.array(degree15_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[0,1,0]])
    X_data_increase.append(degree15_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_300_5.iloc[i+15,0])

for i in range (len(degree15_400_5)-15):
    X_data.append(np.transpose([np.array(degree15_400_5.iloc[i:i+15,1])/step_max,np.array(degree15_400_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,1,0],[0,0,1]])
    X_data_increase.append(degree15_400_5.iloc[i+15,1]/step_max)
    Y_data.append(degree15_400_5.iloc[i+15,0])

########25度###############
for i in range (len(degree25_200_5)-15):
    X_data.append(np.transpose([np.array(degree25_200_5.iloc[i:i+15,1])/step_max,np.array(degree25_200_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,0,1],[1,0,0]])
    X_data_increase.append(degree25_200_5.iloc[i+15,1]/step_max)
    Y_data.append(degree25_200_5.iloc[i+15,0])

for i in range (len(degree25_300_5)-15):
    X_data.append(np.transpose([np.array(degree25_300_5.iloc[i:i+15,1])/step_max,np.array(degree25_300_5.iloc[i:i+15,0])]))
    X_data_factor.append([[0,0,1],[0,1,0]])
    X_data_increase.append(degree25_300_5.iloc[i+15,1]/step_max)
    Y_data.append(degree25_300_5.iloc[i+15,0])

X_data = np.array(X_data)
X_data_increase = np.array(X_data_increase)
Y_data = np.array(Y_data)
X_data_factor = np.array(X_data_factor)

In [ ]:
######GRU############
def Fatigue_GRU():
    History_input = tf.keras.layers.Input(shape=(15,2)) #历史数据，15个点
    Predicted_input_x = tf.keras.layers.Input(shape=(1)) #预测点的x坐标
    Class_curve = tf.keras.layers.Input(shape=(2,3)) #判定曲线类型
    encoder_in = tf.keras.layers.Dense(128)(History_input)
    GRU1 = tf.keras.layers.GRU(128,activation='tanh',return_sequences=True)(encoder_in)
    GRU2 = tf.keras.layers.GRU(128,activation='tanh',return_sequences=False)(GRU1)
    LayerNorm = tf.keras.layers.LayerNormalization()(GRU2)
    Dense1 = tf.keras.layers.Dense(128,activation=tf.nn.leaky_relu)(LayerNorm)
    Densex1 = tf.keras.layers.Dense(64)(Predicted_input_x)
    Onehot1 = tf.keras.layers.Dense(64)(Class_curve)
    Onehot2 = tf.keras.layers.LSTM(64,activation='tanh',return_sequences=False)(Onehot1)
    
    Dense_s1 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(tf.concat([Dense1,Densex1,Onehot2],-1))
    Dense_s2 = tf.keras.layers.Dense(64,activation=tf.nn.leaky_relu)(Dense_s1)
    Output = tf.keras.layers.Dense(1,activation='relu')(Dense_s2) ###输出预测
    
    #固化模型 
    model=tf.keras.models.Model(inputs=[History_input,Predicted_input_x,Class_curve],outputs=Output)   
    #model.summary()
    return model

In [ ]:
model = Fatigue_GRU()
model.compile( optimizer='adam',loss=tf.keras.losses.MeanAbsoluteError())
History = model.fit(x=[X_data,X_data_increase,X_data_factor],y=Y_data,batch_size=4,epochs=100)
model.save_weights("./Fatigue_GRU")

In [ ]:
#####训练########
model = Fatigue_GRU()
model.load_weights("./Fatigue_GRU")
History=[]
Batch_size=4
optimizer = tf.keras.optimizers.Adam(1e-5)
for epoch in range(0,50,1):
    count=0
    Average_loss=0
    MinEpochs = int(len(X_data)/Batch_size)
    new_list = np.array([i for i in range (len(X_data))])
    np.random.shuffle(new_list)
    for j in range(MinEpochs):
        count+=1
        tem_list = new_list[j*Batch_size:(j+1)*Batch_size]
        train_X_data = X_data[tem_list]
        train_X_data_increase = X_data_increase[tem_list]
        train_X_data_factor = X_data_factor[tem_list]
        train_Y_data = Y_data[tem_list]
        with tf.GradientTape() as tape:
            Predicted_Y = model([train_X_data,train_X_data_increase,train_X_data_factor]) 
            Loss = tf.keras.losses.MAE(train_Y_data,Predicted_Y)
            
            gradients = tape.gradient(Loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))  
            Average_loss=Average_loss + tf.reduce_mean(Loss)
    Average_loss = Average_loss/count
    History.append([epoch, Average_loss])
    tf.print("=>Epoch%4d  loss:%4.6f" %(epoch, Average_loss))   

    model.save_weights("./Fatigue_GRU")

In [ ]:
###############for test#################
model  = Fatigue_GRU()
model.load_weights("./Fatigue_GRU")
X_data_test = [] 
X_data_increase_test = []
X_data_factor_test = [] 
Y_data_test = []
for i in range (len(degree25_400_5)-15):
    X_data_test.append(np.transpose([np.array(degree25_400_5.iloc[i:i+15,1])/step_max,np.array(degree25_400_5.iloc[i:i+15,0])]))
    X_data_factor_test.append([[0,0,1],[0,0,1]])
    X_data_increase_test.append(degree25_400_5.iloc[i+15,1]/step_max)
    Y_data_test.append(degree25_400_5.iloc[i+15,0])

X_data_test = np.array(X_data_test) 
X_data_increase_test = np.array(X_data_increase_test)
X_data_factor_test = np.array(X_data_factor_test)
Y_data_test = np.array(Y_data_test)
    
predicted_results=[]
for i in range(len(X_data_test)):
    predicted_results.append(model([X_data_test[i:i+1],X_data_increase_test[i:i+1],X_data_factor_test[i:i+1]]))

In [ ]:
plt.scatter(np.squeeze(Y_data_test),np.squeeze(predicted_results))
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.grid() 
plt.show()

In [ ]:
#####绘制曲线######
model  = Fatigue_GRU()
model.load_weights("./Fatigue_GRU")
initial_data_x = np.zeros(len(degree25_400_5))
initial_data_x[0:20] = degree25_400_5.iloc[0:20,0]
initial_data_inc = degree25_400_5.iloc[:,1]/step_max
for i in range(20,len(initial_data_inc)):
    X_data_test = np.transpose([np.array(degree25_400_5.iloc[i-15:i,1])/step_max,np.array(degree25_400_5.iloc[i-15:i,0])])[np.newaxis,...]
    X_data_increase_test = np.array(degree25_400_5.iloc[i,1]/step_max)[np.newaxis,...]
    X_data_factor_test_test = np.array([[0,0,1],[0,0,1]])[np.newaxis,...]
    initial_data_x[i]= model([X_data_test,X_data_increase_test,X_data_factor_test_test]) 

In [ ]:
plt.plot(initial_data_inc*step_max,initial_data_x)
plt.plot(initial_data_inc*step_max,degree25_400_5.iloc[:,0])
plt.show()
Ontest_set = np.concatenate((np.array(initial_data_inc*step_max)[...,np.newaxis],
                             np.array(initial_data_x)[...,np.newaxis],
                             np.array(degree25_400_5.iloc[:,0])[...,np.newaxis]),-1)
Ontest_set = pd.DataFrame(Ontest_set)
Ontest_set.to_excel("D:/AI in NTU/PINN-fatigue/实验结果汇总/GRU_testset.xlsx")

In [ ]:
###############for test#################
model  = Fatigue_GRU()
model.load_weights("./Fatigue_GRU")
    
predicted_results=[]
for i in range(len(X_data)):
    predicted_results.append(model([X_data[i:i+1],X_data_increase[i:i+1],X_data_factor[i:i+1]]))
    
ON_train_set = np.concatenate((np.squeeze(Y_data)[...,np.newaxis],np.squeeze(predicted_results)[...,np.newaxis]),-1)
ON_train_set = pd.DataFrame(ON_train_set)
ON_train_set.to_excel("D:/AI in NTU/PINN-fatigue/实验结果汇总/GRU_trainset.xlsx")